1.准备

In [ ]:
!wget https://static.leiphone.com/%E7%BE%8E%E9%A3%9F%E6%8C%91%E6%88%98%EF%BC%881%EF%BC%89%E8%B1%86%E8%85%90%E5%92%8C%E5%9C%9F%E8%B1%86.zip    
#下载数据集
#Download dataset
!unzip 美食挑战（1）豆腐和土豆.zip
!rm 美食挑战（1）豆腐和土豆.zip

In [1]:
! wget https://www.cs.toronto.edu/~frossard/vgg16/vgg16_weights.npz
#Download model

--2019-12-06 20:58:48--  https://www.cs.toronto.edu/~frossard/vgg16/vgg16_weights.npz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553436134 (528M)
Saving to: ‘vgg16_weights.npz’

100%[======================================>] 553,436,134 2.39MB/s   in 3m 47s 

2019-12-06 21:02:40 (2.33 MB/s) - ‘vgg16_weights.npz’ saved [553436134/553436134]



2.构建模型

In [1]:
import os
import tensorflow as tf
import numpy as np
from time import time
import VGG16_model as model
import utils
import warnings
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"]='3'
print(tf.__version__)

1.12.0


In [2]:
tf.reset_default_graph()
image_dir="./train"
label_path="train.csv"
batch_size=50
capacity=50
sess=tf.Session()

xs,ys=utils.get_file(image_dir,label_path)
print(xs[0],xs[-1])
image_batch,label_batch=utils.get_batch(xs,ys,224,224,batch_size,capacity)
x=tf.placeholder(tf.float32,[None,224,224,3])
y=tf.placeholder(tf.int32,[None,2])
vgg=model.vgg16(x)
fc8_dinetuining=vgg.probs
loss_function=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc8_dinetuining,labels=y))
prediction=tf.argmax(fc8_dinetuining,1)
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(fc8_dinetuining,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss_function)
sess.run(tf.global_variables_initializer())
vgg.load_weights('vgg16_weights.npz',sess)
saver=tf.train.Saver()

7502
7502
./train/0.jpg ./train/7501.jpg
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.dat

3.训练

In [ ]:
ckpt_dir='./model/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
saver=tf.train.Saver(max_to_keep=3)
ckpt=tf.train.latest_checkpoint(ckpt_dir)
if ckpt!=None:
    saver.restore(sess,ckpt)
else:
    print("Training from scratch!!")

coord=tf.train.Coordinator()
threads=tf.train.start_queue_runners(coord=coord,sess=sess)
epoch_start_time=time()
acc_list=[]
loss_list=[]
for i in range(1000):
    images,labels=sess.run([image_batch,label_batch])
    labels=utils.onehot(labels)
    sess.run(optimizer,feed_dict={x:images,y:labels})
    loss,acc=sess.run([loss_function,accuracy],feed_dict={x:images,y:labels})
    acc_list.append(acc)
    loss_list.append(loss)
    #print("Now the loss is %f"%loss)
    #print("Now the acc is %f"%np.mean(acc_list))
    epoch_end_time=time()
    #print("epoch takes:",(epoch_end_time-epoch_start_time))
    epoch_start_time=epoch_end_time
    if (i+1)%50==0:
        print("Now the loss is %f"%np.mean(loss_list))
        print("Now the acc is %f"%np.mean(acc_list))
        acc_list=[]
        loss_list=[]
        print("----------Epoch %d id Finished --------"%i)
    if (i+1)%500==0:
        saver.save(sess,"./model/tp.ckpt")
    
saver.save(sess,'./model/')
print("Optimizer Finished")
duration=time()-startTime
print("Total takes:{:.2f}".format(duration))
coord.request_stop()#通知其他线程关闭
coord.join(threads)#等待所有线程关闭

4.预测

In [3]:
#预测
import PIL.Image
import pandas as pd
import matplotlib.pyplot as plt
tf.reset_default_graph()
means=[123.68,116.779,103.939]
x=tf.placeholder(tf.float32,[None,224,224,3])
sess=tf.Session()
vgg=model.vgg16(x)
fc8_dinetuining=vgg.probs
saver=tf.train.Saver()
ckpt_dir='./model/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
ckpt=tf.train.latest_checkpoint(ckpt_dir)
if ckpt!=None:
    saver.restore(sess,ckpt)
else:
    print("Training from scratch!!")
print("Model restoring")
#saver.restore(sess,'./model/')#默认恢复最后保存的模型
#saver.restore(sess,'./model/epoch_0008000.ckpt)
#恢复指定检查点的模型
label_list=[]
for i in range(1047):
    img_path='./test/{:}.jpg'.format(i)
    img=PIL.Image.open(img_path)
    img=np.float32(img.resize((224,224),PIL.Image.ANTIALIAS))
    for c in range(3):
        img[:,:,c]-=means[c]
    #print(img)
    imgs=np.array([img],dtype=np.float32)
    #print(imgs.shape)
    prob=sess.run(fc8_dinetuining,feed_dict={x:imgs})
    max_index=np.argmax(prob)
    label_list.append(max_index)
result=np.array(label_list,dtype=np.uint8)
print(result.shape)
print(label_list)
data1 = pd.DataFrame(result)
data1.to_csv('test_label.csv')

INFO:tensorflow:Restoring parameters from ./model/tp.ckpt
Model restoring
(1047,)
[0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,

In [1]:
!touch test_label.csv